# Drum Transcriber (Omnizart Edition) Colab - Robust Install

Run this notebook to launch the **SOTA Omnizart-powered** Drum Transcriber Web UI.

**Note**: This version uses a robust installation strategy to bypass dependency conflicts on Colab.

In [ ]:
# @title 1. Setup Environment (Robust Fix)
# @markdown This cell installs library dependencies with robust fixes for Python 3.10 and dependency overrides.

import os
import sys

# 1. Install System Dependencies
!sudo apt-get install libsndfile1 ffmpeg -y

# 2. Fix Build Tools
!pip install --upgrade pip setuptools wheel
!pip install cython numpy==1.23.5

# 3. Install Madmom from Git (3.10 compatible version)
!pip install git+https://github.com/CPJKU/madmom.git

# 4. Install other base dependencies
!pip install pretty_midi gradio yt-dlp librosa==0.8.1 pandas plotly scikit-learn soundfile jsonschema

# 5. Clone and Patch Omnizart
if not os.path.exists("omnizart"):
    !git clone https://github.com/Music-and-Culture-Technology-Lab/omnizart.git

%cd omnizart
# Patch versions and requirements
!sed -i 's/python_requires=".*"/python_requires=">=3.8"/' setup.py
!sed -i 's/PyYAML<6.0.0,>=5.3.1/PyYAML>=5.3.1/' setup.py
!sed -i 's/numba==0.48/numba>=0.48/' setup.py
!sed -i 's/librosa==0.8.0/librosa>=0.8.0/' setup.py
!sed -i 's/madmom==0.16.1/madmom>=0.16.1/' setup.py

# Install Omnizart ignoring dependency checks to bypass the madmom build error
!pip install --no-deps .
%cd ..

# 6. Clone DrumTranscriber Repo
repo_url = "https://github.com/AgentHitmanFaris/DrumTranscriber.git"
repo_name = "DrumTranscriber_App"

if not os.path.exists(repo_name):
    !git clone {repo_url} {repo_name}

%cd {repo_name}

# Refresh sys.path to find newly installed packages
import site
from importlib import reload
reload(site)

# 7. Download Omnizart checkpoints
try:
    import omnizart
    from omnizart.remote import download_checkpoints
    download_checkpoints()
    print("Omnizart checkpoints downloaded successfully.")
except Exception as e:
    print(f"Error importing omnizart: {e}")
    print("Trying direct download via python command...")
    !python3 -c "from omnizart.remote import download_checkpoints; download_checkpoints()"


In [ ]:
# @title 2. Run App (Omnizart Mode)
# @markdown This will launch the Gradio interface using the Omnizart model.

launcher_code = """
import os
import gradio_app
from omnizart_wrapper import OmnizartWrapper

# Patch the load_model function to return OmnizartWrapper
def load_omnizart():
    print('Loading Omnizart Wrapper...')
    return OmnizartWrapper()

gradio_app.load_model = load_omnizart

# Run the demo
gradio_app.demo.launch(share=True)
"""

with open('run_omnizart.py', 'w') as f:
    f.write(launcher_code)

!python run_omnizart.py
